In [1]:
import pandas as pd
import os
import shutil
import cv2
from tqdm import tqdm

In [3]:
# dfOriginal = pd.read_csv('D:/Datasets/celeba-dataset/list_attr_celeba.csv')
dfOriginal = pd.read_csv('C:/datasets/celeba-dataset/list_attr_celeba.csv')

dfOriginal.head(10)

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
5,000006.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,1,-1,1,-1,-1,1
6,000007.jpg,1,-1,1,1,-1,-1,1,1,1,...,-1,-1,1,-1,-1,-1,-1,-1,-1,1
7,000008.jpg,1,1,-1,1,-1,-1,1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
8,000009.jpg,-1,1,1,-1,-1,1,1,-1,-1,...,-1,1,-1,-1,1,-1,1,-1,-1,1
9,000010.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,1


In [4]:
dfOriginal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202599 entries, 0 to 202598
Data columns (total 41 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   image_id             202599 non-null  object
 1   5_o_Clock_Shadow     202599 non-null  int64 
 2   Arched_Eyebrows      202599 non-null  int64 
 3   Attractive           202599 non-null  int64 
 4   Bags_Under_Eyes      202599 non-null  int64 
 5   Bald                 202599 non-null  int64 
 6   Bangs                202599 non-null  int64 
 7   Big_Lips             202599 non-null  int64 
 8   Big_Nose             202599 non-null  int64 
 9   Black_Hair           202599 non-null  int64 
 10  Blond_Hair           202599 non-null  int64 
 11  Blurry               202599 non-null  int64 
 12  Brown_Hair           202599 non-null  int64 
 13  Bushy_Eyebrows       202599 non-null  int64 
 14  Chubby               202599 non-null  int64 
 15  Double_Chin          202599 non-nu

In [47]:
df = dfOriginal

canBeUsed = [
    'image_id',
    'Arched_Eyebrows',
    'Attractive',
    'Bags_Under_Eyes',
    'Big_Lips',
    'Big_Nose',
    'Blurry',
    'Bushy_Eyebrows',
    'Chubby',
    'Double_Chin',
    'Eyeglasses',
    'High_Cheekbones',
    'Male', 
    'Mouth_Slightly_Open',
    'Narrow_Eyes',
    'Oval_Face',
    'Pointy_Nose',
    'Smiling',
    'Young'
]

dfResult = pd.DataFrame()
register = {}

df = df[canBeUsed]

df = df[
    (df['Male'] == -1)        &   \
    (df['Blurry'] == -1)      &   \
    (df['Eyeglasses'] == -1)  &   \
    (df['Young'] == 1)]
df = df.drop(['Male' , 'Blurry', 'Eyeglasses', 'Young'] , axis='columns')

for col in df.columns: 
#     print(col)
    dfTemp = df[df[col] == 1]    
        
    dfTemp = dfTemp[['image_id', col]]
#     print(f'{col}: {len(dfTemp)}')
    
    register['col'] = col
    register['length'] = len(dfTemp)    
    dfResult = dfResult.append(register, ignore_index=True)
    
dfResult.sort_values(by=['length'], ascending=False)

,col,length
2,Attractive,73910.0
9,High_Cheekbones,53497.0
14,Smiling,51258.0
10,Mouth_Slightly_Open,49856.0
1,Arched_Eyebrows,42949.0
13,Pointy_Nose,36540.0
12,Oval_Face,34963.0
4,Big_Lips,30430.0
11,Narrow_Eyes,10334.0
3,Bags_Under_Eyes,8941.0


In [29]:
# # https://kanoki.org/2020/03/09/how-to-use-pandas-count-and-value_counts/
# # Pandas Value Count for Multiple Columns

# # df.count(0)
# df.apply(pd.Series.value_counts, axis=0)
# # df1 = df.melt(var_name='columns', value_name='values')
# # pd.crosstab(index=df1['values'], columns=df1['columns'])

In [48]:
df = dfOriginal

# Faster way # https://stackoverflow.com/questions/22086116/how-do-you-filter-pandas-dataframes-by-multiple-columns
df = df[
    (df['Male'] == -1)        &   \
    (df['Blurry'] == -1)      &   \
    (df['Eyeglasses'] == -1)  &   \
    (df['Young'] == 1)
]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97609 entries, 0 to 202598
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   image_id             97609 non-null  object
 1   5_o_Clock_Shadow     97609 non-null  int64 
 2   Arched_Eyebrows      97609 non-null  int64 
 3   Attractive           97609 non-null  int64 
 4   Bags_Under_Eyes      97609 non-null  int64 
 5   Bald                 97609 non-null  int64 
 6   Bangs                97609 non-null  int64 
 7   Big_Lips             97609 non-null  int64 
 8   Big_Nose             97609 non-null  int64 
 9   Black_Hair           97609 non-null  int64 
 10  Blond_Hair           97609 non-null  int64 
 11  Blurry               97609 non-null  int64 
 12  Brown_Hair           97609 non-null  int64 
 13  Bushy_Eyebrows       97609 non-null  int64 
 14  Chubby               97609 non-null  int64 
 15  Double_Chin          97609 non-null  int64 
 16  Eye

In [5]:
dfTemp = df[df['Arched_Eyebrows'] == 1]
dfTemp = dfTemp[['image_id', 'Arched_Eyebrows']]
print(f'Arched_Eyebrows: {len(dfTemp)}')

dfTemp = df[df['Bags_Under_Eyes'] == 1]
dfTemp = dfTemp[['image_id', 'Bags_Under_Eyes']]
print(f'Bags_Under_Eyes: {len(dfTemp)}')

dfTemp = df[df['Big_Lips'] == 1]
dfTemp = dfTemp[['image_id', 'Big_Lips']]
print(f'Big_Lips: {len(dfTemp)}')

dfTemp = df[df['Big_Nose'] == 1]
dfTemp = dfTemp[['image_id', 'Big_Nose']]
print(f'Big_Nose: {len(dfTemp)}')

dfTemp = df[df['High_Cheekbones'] == 1]
dfTemp = dfTemp[['image_id', 'High_Cheekbones']]
print(f'High_Cheekbones: {len(dfTemp)}')

dfTemp = df[df['Narrow_Eyes'] == 1]
dfTemp = dfTemp[['image_id', 'Narrow_Eyes']]
print(f'Narrow_Eyes: {len(dfTemp)}')

dfTemp = df[df['Oval_Face'] == 1]
dfTemp = dfTemp[['image_id', 'Oval_Face']]
print(f'Oval_Face: {len(dfTemp)}')

dfTemp = df[df['Smiling'] == 1]
dfTemp = dfTemp[['image_id', 'Smiling']]
print(f'Smiling: {len(dfTemp)}')

Arched_Eyebrows: 42949
Bags_Under_Eyes: 8941
Big_Lips: 30430
Big_Nose: 8516
High_Cheekbones: 53497
Narrow_Eyes: 10334
Oval_Face: 34963
Smiling: 51258


In [6]:
dfTemp = df[df['High_Cheekbones'] == 1]
dfTemp = dfTemp[['image_id', 'High_Cheekbones']]
display(dfTemp.head())
display(dfTemp.tail())

,image_id,High_Cheekbones
0,000001.jpg,1
1,000002.jpg,1
8,000009.jpg,1
9,000010.jpg,1
13,000014.jpg,1


,image_id,High_Cheekbones
202586,202587.jpg,1
202590,202591.jpg,1
202591,202592.jpg,1
202592,202593.jpg,1
202597,202598.jpg,1


In [7]:
# CSV_FILE_NAME = 'celeba-mulheres-High_Cheekbones.csv'
# CSV_FOLDER = './csv'
# CSV_FILE_PATH = f'{CSV_FOLDER}/{CSV_FILE_NAME}'

In [8]:
# df.to_csv(
#     CSV_FILE_PATH, 
#     encoding='utf-8',
#     index=False)

In [9]:
CSV_FOLDER = './csv'

def saveCsv(df, attribute):
    
    CSV_FILE_NAME = f'celeba-women-{attribute}.csv'
    CSV_FILE_PATH = f'{CSV_FOLDER}/{CSV_FILE_NAME}'
    
    df.to_csv(
        CSV_FILE_PATH, 
        encoding='utf-8',
        index=False)

In [10]:
IMAGES_CELEBA_FOLDER = 'D:/Datasets/celeba-dataset/img_align_celeba/'

def getImages(df, attribute):
    
    destinationFolder = f'./images/originals--{attribute}/'
    try:
        shutil.rmtree(destinationFolder)
        os.mkdir(destinationFolder)
    except OSError as e:
        os.mkdir(destinationFolder)
    
    counter = 0
    for file in tqdm(df['image_id']):
        loadedImage = cv2.imread( IMAGES_CELEBA_FOLDER + file )
        cv2.imwrite( destinationFolder + file, loadedImage )
        counter += 1

In [ ]:
# All
# attributes = [
#     'Arched_Eyebrows',
#     'Bags_Under_Eyes',
#     'Big_Lips',
#     'Big_Nose',
#     'High_Cheekbones',
#     'Narrow_Eyes',
#     'Oval_Face',
#     'Smiling']

# Just High_Cheekbones
# attributes = ['High_Cheekbones']

# All without High_Cheekbones
# attributes = [
#     'Arched_Eyebrows',
#     'Bags_Under_Eyes',
#     'Big_Lips',
#     'Big_Nose',
#     'Narrow_Eyes',
#     'Oval_Face',
#     'Smiling']

print('Processing...')
for attribute in attributes:
    # print(attribute)
    
    df = dfOriginal
    df = df[
        (df['Male'] == -1)        &   \
        (df['Blurry'] == -1)      &   \
        (df['Eyeglasses'] == -1)  &   \
        (df['Young'] == 1)
    ]
    df = df[df[attribute] == 1]
    df = df[['image_id', attribute]]
    print(f'{attribute}: {len(df)}')
    
    saveCsv(df, attribute)
    getImages(df, attribute)

print( '\n----------------------------------------------------' )
print( 'Finalized' )